# Проект: Рынок заведений общественного питания Москвы

Инвесторы из фонда «Shut Up and Take My Money» решили расширить свои горизонты и открыть заведение в Москве. Мне, в роли аналитика, нужно подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего заказчику места.

# Описание данных

Файл moscow_places.csv:
- name — название заведения;  
- address — адрес заведения;
- category — категория заведения, например «кафе», «пиццерия» или «кофейня»;
- hours — информация о днях и часах работы;
- lat — широта географической точки, в которой находится заведение;
- lng — долгота географической точки, в которой находится заведение;
- rating — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);
- price — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;
- avg_bill — строка, которая хранит среднюю стоимость заказа в виде диапазона, например:  
    1. «Средний счёт: 1000–1500 ₽»;
    2. «Цена чашки капучино: 130–220 ₽»;
    3. «Цена бокала пива: 400–600 ₽».
    и так далее;
- middle_avg_bill — число с оценкой среднего чека, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Средний счёт»:
    1. Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.
    2. Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.
    3. Если значения нет или оно не начинается с подстроки «Средний счёт», то в столбец ничего не войдёт.
- middle_coffee_cup — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»:
    1. Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.
    2. Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.
    3. Если значения нет или оно не начинается с подстроки «Цена одной чашки капучино», то в столбец ничего не войдёт.
- chain — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки):
    1. 0 — заведение не является сетевым
    2. 1 — заведение является сетевым
- district — административный район, в котором находится заведение, например Центральный административный округ;
- seats — количество посадочных мест.

# План работы

**Шаг 1. Загрузка данных и изучение общей информации   
Шаг 2. Выполнение предобработки данных  
Шаг 3. Анализирование данных  
Шаг 4. Детализирование исследования: открытие кофейни  
Шаг 5. Подготовление презентации** 

1. [Общая информация о данных](#start)
2. [Предобработка данных](#2)
3. [Анализ данных](#3)
    * [Визуализация количества объектов общественного питания по категориям](#4)
    * [Визуализация количества посадочных мест в местах по категориям](#5)
    * [Изображение соотношение сетевых и несетевых заведений в датасете](#6)
    * [Иллюстрация сетевых заведений](#7)
    * [Визуализация топ-15 популярных сетей в Москве](#8)
    * [Изображение общего количества заведений и количества заведений каждой категории по районам](#9)
    * [Визуализирование распределения средних рейтингов по категориям заведений](#10)
    * [Построение фонофой картограммы среднего рейтинга заведений по районам](#11)
    * [Отображение всех заведений датасета на карте с помощью кластеров](#12)
    * [Пятнадцать улиц с наибольшим количеством заведений](#13)
    * [Нахождение улиц, на которых находится только один объект общепита](#14)
    * [Построение хороплета с медианной оценкой среднего чека](#15)
    * [Иллюстрация других взаимосвязей](#16)
4. [Детализированное исследование: открытие кофейни](#17)

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from folium import Map, Marker, Choropleth
from folium.plugins import MarkerCluster
from datetime import datetime, timedelta
from plotly import graph_objects as go

ModuleNotFoundError: No module named 'folium'

### Общая информация о данных
<a id="start"></a> 

In [ ]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/moscow_places.csv')
display(data.info())

**Вывод:** в датасете представлено 8406 заведений. Столбцы отображают особенности каждого ресторана, оформлены в змеином стиле с прописной буквы. В них хранятся данные в строковом и числовых форматах. 

### Предобработка данных
<a id="2"></a> 

In [ ]:
def info_duplicates(*dataframes):
    for df in dataframes:
        df.info()
        print()
        print('\033[1m' + 'Количество дубликатов в таблице:' + '\033[0m', df.duplicated().sum())
        print()       

In [ ]:
info_duplicates(data)
print('\033[1m' + 
      f'Доля пропущенных значений от общего числа в столбце hours: {round(data.hours.isna().sum() / data.shape[0] * 100, 2)}%'
      + '\033[0m')

In [ ]:
def make_street_from_adress(cell):
    cell = cell.replace('Москва,', '').strip().split(',')
    return cell[0]

In [ ]:
data = data.dropna(subset = ['hours']) 
data['street'] = data.address.apply(make_street_from_adress)
data['is_24_7'] = data.hours.str.contains(r'\ежедневно,круглосуточно')
data[['price', 'avg_bill']] = data[['price', 'avg_bill']].fillna('неизвестно')
data[['seats']] = data[['seats']].fillna(0)
display(data.tail(), data.info())

In [ ]:
print('\033[1m' + 'При поиске неявных дубликатов по ключевым параметрам, их число составило:' + '\033[1m', 
      data.duplicated(subset=['name', 'category', 'address']).sum())
print('Если привести столбец названий в нижний регистр, то обнаружится такое число дубликатов:', data.name.str.lower().duplicated().sum())

**Вывод:** в данных дубликатов нет. Пропуски встречаются в столбцах hours, price, avg_bill, middle_avg_bill, middle_coffee_cup и seats. Что касается столбца hours, то его доля пропущенных значений допустима для удаления.  Так как пропущенных значений слишком много, поставил на их место заглушки. Создал столбец с названиями улиц, и ещё один с обозначением, что заведение работает ежедневно и круглосуточно. Для поиска неявных дубликатов необходимо определить ключевые параметры. Я выбрал название, категорию и адрес. По такому подмножеству неявных дубликатов не было выявлено. При переводе в нижний регистр названий заведений и подсчёте дубликатов, их окажется 2651, что не удивительно, в наборе данных представлены сетевые заведения, которые имеют одинаковые названия.

### Анализ данных
<a id="3"></a> 

#### Визуализация количества объектов общественного питания по категориям
<a id="4"></a> 

In [ ]:
plt.figure(figsize=(15, 5));
sns.set_palette('flare', 8)
sns.barplot(x='index', y='category', data= data.category.value_counts().reset_index())
plt.xlabel('Название заведения', fontsize = 12);
plt.ylabel('Число встречаемости', fontsize = 12);
plt.xticks(rotation=45, fontsize = 15);
plt.title('Распределение заведений по категориям', fontsize = 20);

**Вывод:** топ-3 категорий по числу заведений таковы: кафе, рестораны, кофейни.

####  Визуализация количества посадочных мест в местах по категориям
<a id="5"></a> 

In [ ]:
plt.figure(figsize=(15, 5));
sns.barplot(x='category', y='seats', data=data.groupby('category').seats.median().reset_index().sort_values(by = 'seats'));
plt.xlabel('Название заведения', fontsize = 12);
plt.ylabel('Количество мест', fontsize = 12);
plt.xticks(rotation=45, fontsize = 15);
plt.title('Распределение заведений по количеству мест по категориям', fontsize = 20);

**Вывод:** большего всего посадочных мест находится в ресторанах, пиццериях и барах с пабами. Меньше всего в кафе и столовых. 

#### Изображение соотношения сетевых и не сетевых заведений в датасете
<a id="6"></a> 

In [ ]:
chain = ['Не сетевые', 'Сетевые']
values = data.chain.value_counts().values
fig_pie = go.Figure(data=[go.Pie(labels=chain, values=values)])
fig_pie.show() 

**Вывод:** больше несетевых заведений.

#### Иллюстрация сетевых заведений
<a id="7"></a> 

In [ ]:
plt.figure(figsize=(15, 5));
sns.set_palette('crest', 8)
sns.barplot(x='index', y='category', data=data.query('chain == 1').category.value_counts().reset_index())
plt.xlabel('Название заведения', fontsize = 12);
plt.ylabel('Число встречаемости', fontsize = 12);
plt.xticks(rotation=45, fontsize = 15);
plt.title('Распределение сетевых заведений', fontsize = 20);

**Вывод:** чаще всего сетевыми являются кофейни, рестораны и кафе.

#### Визуализация топ-15 популярных сетей в Москве
<a id="8"></a> 

In [ ]:
plt.figure(figsize=(20, 10));
sns.set_palette('rocket', 15)
sns.barplot(x='name', y='index', data=data.loc[data.chain == 1].name.value_counts().head(15).reset_index())
plt.xlabel('Число встречаемости', fontsize = 12);
plt.ylabel('Название заведения', fontsize = 12);
plt.yticks(fontsize = 15);
plt.title('Топ-15 популярных сетей в Москве', fontsize = 20);

**Вывод:** мне знакомы большинство из этих сетей. В каждой из сетей не менее 20 заведений. Они относятся к следующим категориям заведений: булочная, пиццерия, кофейня, кафе, ресторан, быстрое питание, столовая, бар,паб.

#### Изображение общего количества заведений и количества заведений каждой категории по районам
<a id="9"></a> 

In [ ]:
print('Такие административные районы Москвы присутствуют в датасете: {}'.format(data.district.unique()))

In [ ]:
districts_categories = data.groupby(['district', 'category']).agg({'name': 'count'}).reset_index()
districts_categories.loc[72] = ['Oбщее количество заведений', 'Все', data.shape[0]]
fig = px.bar(districts_categories,  x='name', y='district', color='category'
             , title='Общее количество заведений и количество заведений каждой категории по районам')
fig.update_layout(xaxis_title="Количество заведений",
                  yaxis_title="Административный район")
fig.show()

#### Визуализирование распределения средних рейтингов по категориям заведений
<a id="10"></a> 

In [ ]:
mean_rating = data.groupby('category').rating.mean().reset_index()
mean_rating.rating = round(mean_rating.rating, 3)
plt.figure(figsize=(15, 5));
sns.barplot(x='category', y='rating', data=mean_rating)
plt.xlabel('Название заведения', fontsize = 12);
plt.ylabel('Средний рейтинг', fontsize = 12);
plt.yticks(fontsize = 15);
plt.title('Распределение средних рейтингов по категориям', fontsize = 20);

**Вывод:** усреднённые рейтинги в разных типах общепита различаются слабо.

#### Построение фонофой картограммы среднего рейтинга заведений по районам
<a id="11"></a> 

In [ ]:
rating_df = data.groupby('district').rating.mean()
moscow_lat, moscow_lng = 55.751244, 37.618423
m_choropleth_rating = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles="Cartodb Positron")
Choropleth(
    geo_data = '/datasets/admin_level_geomap.geojson',
    data = rating_df,
    columns = ['district', 'rating'],
    key_on = 'feature.name',
    fill_color = 'YlOrRd',
    fill_opacity = 0.8,
    legend_name = 'Средний рейтинг заведений по районам'
).add_to(m_choropleth_rating)
m_choropleth_rating

#### Отображение всех заведений датасета на карте с помощью кластеров
<a id="12"></a> 

In [ ]:
m_clusters = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles="Cartodb Positron")
marker_cluster = MarkerCluster().add_to(m_clusters)

In [ ]:
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup = f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

In [ ]:
data.apply(create_clusters, axis=1)
m_clusters

#### Пятнадцать улиц с наибольшим количеством заведений
<a id="13"></a> 

In [ ]:
steet_amount = data.street.value_counts().reset_index().head(15)
steet_amount.columns = ['Название улицы', 'Количество заведений']
print('\033[1m' + 'Топ-15 улиц по количеству заведений:' + '\033[1m')
display(steet_amount)

In [ ]:
streets_categories = (
    data.query("street.isin(@steet_amount['Название улицы'])")
    .groupby(['street', 'category'])
    .agg({'name': 'count'}).reset_index()
)
fig_1 = px.bar(streets_categories,  x='name', y='street', color='category'
             , title='Распределение количества заведений и их категорий по этим улицам')
fig_1.update_layout(xaxis_title="Количество заведений",
                    yaxis_title="Название улицы")
fig_1.show()

#### Нахождение улиц, на которых находится только один объект общепита
<a id="14"></a> 

In [ ]:
one_catering = data.street.value_counts().reset_index().query('street == 1')
one_catering.columns = ['Название улицы', 'Число заведений']
display(one_catering)
print('\033[1mЧисло улиц с одним заведением: {}\033[1m'.format(one_catering.shape[0]))
print()
print('Единичные заведения по категориям:')
display(data.query('index.isin(@one_catering.index)').category.value_counts())

**Вывод:** Количество улиц, на которых расположено одно заведение - 471. По категориям они распределяются также как и самые популярные во всем датасете.

#### Построение хороплета с медианной оценкой среднего чека
<a id="15"></a> 

In [ ]:
district_middle_avg_bill = data.groupby('district').middle_avg_bill.median()
m_choropleth_middle_avg_bill = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles="Cartodb Positron")
Choropleth(
    geo_data = '/datasets/admin_level_geomap.geojson',
    data = district_middle_avg_bill,
    columns = ['district', 'middle_avg_bill'],
    key_on = 'feature.name',
    fill_color = 'YlOrRd',
    fill_opacity = 0.8,
    legend_name = 'Медианая оценка среднего чека'
).add_to(m_choropleth_middle_avg_bill)
m_choropleth_middle_avg_bill

**Вывод:** в среднем цены по районам начинаются от 450 до 1000 рублей. Чем западнее расположен район, тем выше цены.

#### Иллюстрация других взаимосвязей
<a id="16"></a> 

In [ ]:
def only_hours(cell):
    def interval_to_number(time):
        start_time_str, end_time_str = time.split('–')
        start_time = datetime.strptime(start_time_str, '%H:%M')
        end_time = datetime.strptime(end_time_str, '%H:%M')
        if end_time < start_time:
            end_time += timedelta(days=1)
        duration = end_time - start_time
        time = round(duration.total_seconds() / 3600)
        return time
    try:
        if 'круглосуточно' in cell:
            cell = 24
            return cell
        else:
            trans_table = {ord(',') : None, ord('-') : None}
            cell = ''.join(i for i in cell if not i.isalpha()).translate(trans_table).strip().replace(' ', '')
            if ';' not in cell:
                return interval_to_number(cell)
            else:
                cell = cell.split(';')
                median_hour = 0
                for interval in cell:
                    median_hour += interval_to_number(interval)
                cell = round(median_hour / len(cell))
                return cell
    except:
        cell = None
        return cell
#создал функцию, которая из дней недели и интервалов времени преобразует в число часов работы заведений.
#есть недоработки: не учтены часы перерыва, столбцы со строкой "круглосуточно" переделаны в 24, даже там где не все дни недели
#заведение работает полные сутки. Есть незначительные потери, но думаю суть часов работы заведений отражает в полной мере.
#также в строках, где 2 или 3 варианта времени работы по разным дням, вычеслено среднее значение.

In [ ]:
data['only_hours'] = data.hours.apply(only_hours)
print('\033[1m' + 'Потеряно строк после обработки:' + '\033[1m', data.query('only_hours.isna()').shape[0])

In [ ]:
district_hours = data.groupby(['district', 'only_hours']).agg({'name': 'count'}).reset_index()
fig_2 = px.bar(district_hours,  x='name', y='district', color='only_hours', 
               title='Распределение числа заведений по часам работы и по районам')
fig_2.update_layout(xaxis_title="Количество заведений",
                    yaxis_title="Название района")
fig_2.show()

In [ ]:
category_hours = data.groupby(['category', 'only_hours']).agg({'name': 'count'}).reset_index()
fig_3 = px.bar(category_hours,  x='name', y='category', color='only_hours', 
               title='Распределение числа заведений и их часам работы по категориям')
fig_3.update_layout(xaxis_title="Количество заведений",
                    yaxis_title="Категория")
fig_3.show()

**Вывод:** во всех районах и во всех категориях оказались, как ни странно, круглосуточные магазины. По всем районам чаще всего заведения работают по 12 часов. Примерно такое же положение обстоит и в разбивке по категориям, кроме столовых, они зачастую работают по 8 часов.

In [ ]:
print(f'Обозначу за плохой рейтинг: {data.rating.quantile(0.25)} и меньше, что составляет от всех рейтингов 25%')

In [ ]:
low_ratings = data.query('rating <= 4.1').groupby(['category', 'rating']).middle_avg_bill.mean().reset_index()
fig_4 = px.bar(low_ratings,  x='middle_avg_bill', y='category', color='rating', 
               title='Распределение усреднённых чеков и их рейтингов по категориям')
fig_4.update_layout(xaxis_title="Усреднённый чек",
                    yaxis_title="Категория")
fig_4.show()

**Вывод:** самые низкие рейтинги относятся к заведениям кафе и быстрого питания. В общем по всем категориям нет зависимости между низким рейтингом и средним чеком. Встречаются как высокие средние чеки с низким рейтингом, так и высокие рейтинги с небольшим чеком.

**Общий вывод блока:** Топ-3 категории по количеству заведений включают кафе, рестораны и кофейни. Наибольшее количество посадочных мест наблюдается в ресторанах, кафе и кофейнях, а наименьшее — в булочных и столовых. В датасете представлено больше несетевых заведений, а сетевыми чаще всего являются кофейни, рестораны и кафе. Мне известны большинство из этих сетей, в каждой из которых насчитывается не менее 20 заведений. Они принадлежат следующим категориям: булочная, пиццерия, кофейня, кафе, ресторан, фастфуд, столовая, бар и паб. Усредненные рейтинги для различных типов общепита почти не отличаются. Количество улиц с одним заведением составляет 471, и распределение по категориям совпадает с самыми популярными в датасете. В среднем цены по районам варьируются от 450 до 1000 рублей: чем дальше на запад расположен район, тем выше цены. В каждом районе и в каждой категории, неожиданно, присутствуют круглосуточные магазины. В большинстве районов заведения, как правило, работают по 12 часов. Аналогичная ситуация наблюдается и среди категорий, за исключением столовых, которые чаще всего функционируют в течение 8 часов. Наименьшие рейтинги принадлежат заведениям кафетериев и быстрого питания. В целом, между низким рейтингом и средним чеком среди всех категорий нет явной связи. Можно встретить как высокие средние чеки при низком рейтинге, так и высокие рейтинги при скромном чеке.

### Детализированное исследование: открытие кофейни
<a id="17"></a> 

In [ ]:
print('\033[1m' + f'В датасете представлено {data.loc[data.category == "кофейня"].shape[0]} кофеен' + '\033[1m')
print()
print('\033[1m' + 'Больше всего кофеен в этих районах:' + '\033[1m')
display(
    data.loc[data.category == "кофейня"]
    .groupby('district').category.count()
    .reset_index().sort_values('category', ascending=False).head(3)
)

In [ ]:
amount_coffee_houses = data.loc[data.category == "кофейня"].groupby('district').category.count()
m_choropleth_coffee = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles="Cartodb Positron")
Choropleth(
    geo_data = '/datasets/admin_level_geomap.geojson',
    data = amount_coffee_houses,
    columns = ['district', 'category'],
    key_on = 'feature.name',
    fill_color = 'YlOrRd',
    fill_opacity = 0.8,
    legend_name = 'Количество кофеен по районам'
).add_to(m_choropleth_coffee)
m_choropleth_coffee

In [ ]:
print(
    '\033[1m'+'Кофеен, работающих круглосуточно:'+'\033[1m', 
    data.loc[data.category == "кофейня"].hours.str.contains(r'\круглосуточно').sum())

In [ ]:
rating_coffee_house = data.loc[data.category == "кофейня"].groupby(['district', 'rating']).name.count().reset_index()
fig_5 = px.bar(rating_coffee_house,  x='name', y='district', color='rating', 
               title='Распределение количества кофеен и их рейтингов по районам')
fig_5.update_layout(xaxis_title="Количество кофеен",
                    yaxis_title="Район")
fig_5.show()

In [ ]:
rating_coffee_cup = data.loc[data.category == "кофейня"].groupby(['rating']).middle_coffee_cup.mean().reset_index()
fig_6 = px.bar(rating_coffee_cup,  x='rating', y='middle_coffee_cup', 
               title='Распределение усреднённых стоимостей чашек капучино по рейтингу')
fig_6.update_layout(xaxis_title="Рейтинг",
                    yaxis_title="Средняя стоимость чашки капучино")
fig_6.show()
district_coffee_cup = data.loc[data.category == "кофейня"].groupby(['district']).middle_coffee_cup.mean().reset_index()
fig_7 = px.bar(district_coffee_cup,  x='middle_coffee_cup', y='district', 
               title='Распределение усреднённых стоимостей чашек капучино по районам')
fig_7.update_layout(xaxis_title="Стоимость чашки капучино",
                    yaxis_title="Название района")
fig_7.show()

**Вывод:** в наборе данных представлено 1398 кофеен. Около трети из них располагается в Центральном административном округе. Выявлено 76 кофейнь, работающих круглосуточно. Рейтинги у кофеен варьируются от 2 до 5, хотя ниже 4 встречается нечасто; большинство оценок высокие. Стоит ориентироваться на стоимость чашки капучино от 150 до 200 рублей, так как самые высокие рейтинги получают заведения с таким ценовым диапазоном. Также в разбивке по районам средней стоимости чашки, встречаются значения, представленные выше. На мой взгляд, при открытии кофейни я бы не рассматривал Центральный административный округ из-за большой концентрации этих заведений и, соответственно, высокой конкуренции. Стоит обратить внимание на Северо-Западный административный округ, где небольшое число заведений и преимущественно высокие оценки.

Презентация: <https://disk.yandex.ru/i/CYJgSt6_iCIo4A>